In [1]:
import numpy as np
import os
import time
import argparse
import random

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Input,
    Concatenate,
    Flatten,
    BatchNormalization,
    Activation,
)
from tensorflow.keras.optimizers import Adam
from qkeras import QActivation, QDense, QConv1D, QConv2D, quantized_bits
from qkeras.autoqkeras.utils import print_qmodel_summary

from sklearn.metrics import accuracy_score
from tensorflow_model_optimization.python.core.sparsity.keras import pruning_wrapper


from node_edge_projection import NodeEdgeProjection

Using TensorFlow backend


In [3]:
jetConstituent = np.load("data/jetConstituent_train_150_16f.npy")
target = np.load("data/jetConstituent_train_target_150_16f.npy")

In [4]:
# Restrict the number of constituents to a maximum of NMAX
nmax = 50
jetConstituent = jetConstituent[:, 0:nmax, :]

In [5]:
# The dataset is N_jets x N_constituents x N_features
njet = jetConstituent.shape[0]
nconstit = jetConstituent.shape[1]
nfeat = jetConstituent.shape[2]


print("Number of jets =", njet)
print("Number of constituents =", nconstit)
print("Number of features =", nfeat)


Number of jets = 620000
Number of constituents = 50
Number of features = 16


In [6]:

# nomilization
interquantile_range_30 = [100.17132396698003, 100.41209926605225, 113.84440898895267, 165.31627578735362, 
                          0.11864765174686911, 124.32202949523926, 0.1189630240201951, 2.4549999237060547,
                          0.271871916204691, 0.15521225892007368, 5.597837924957275, 0.27236732020974164, 
                          0.3221405863761902, 0.2523181140422821, 1.6837052702903748, 0.15490140579640882]

interquantile_range_50 = [64.10330410003655, 64.31864662170406, 70.29069595336911, 117.4136219024657, 
                          0.08511629104614249, 89.21204452514644, 0.08535529710352419, 2.265776056051254, 
                          0.2656875118613242, 0.1704555112868547, 5.398193371295928, 0.26594679132103916, 
                          0.31737845018506045, 0.2557373657822608, 1.6240106284618379, 0.17004398591816422]

if nmax==30:
    jetConstituent_norm     = jetConstituent     / interquantile_range_30
    #jetConstituent_val = jetConstituent_val / interquantile_range_30
elif nmax==50:
    jetConstituent_norm     = jetConstituent     / interquantile_range_50
    #jetConstituent_val = jetConstituent_val / interquantile_range_50

print (jetConstituent_norm.max(axis=0))
#print (jetConstituent_val.max(axis=0))

[[2.35960130e+01 2.14541532e+01 4.62176586e+01 2.89219866e+01
  1.05084746e+01 1.74773589e+01 1.04676774e+01 1.10558656e+00
  1.35506682e+00 0.00000000e+00 5.81963807e-01 1.41386344e+00
  0.00000000e+00 1.49971658e+00 6.07598799e-01 3.60097079e-16]
 [1.13870107e+01 1.00242323e+01 2.03236948e+01 1.26879905e+01
  5.38045240e+00 8.79148345e+00 5.36154256e+00 1.10816986e+00
  1.48503529e+00 5.84443111e-01 5.81969725e-01 1.33798981e+00
  3.13419366e-01 1.64620873e+00 6.07693160e-01 5.83927056e-01]
 [5.79905215e+00 5.22570242e+00 1.33489694e+01 8.15498207e+00
  3.49071760e+00 4.32073062e+00 3.49146181e+00 1.10513104e+00
  1.45784469e+00 2.83011030e+00 5.81968002e-01 1.41317505e+00
  1.14628538e+00 1.61435893e+00 6.07582063e-01 2.63561186e+00]
 [3.59554688e+00 4.20843268e+00 1.23604693e+01 7.55125222e+00
  2.48701396e+00 3.23912808e+00 2.48083437e+00 1.11105684e+00
  1.44366106e+00 2.81490645e+00 5.81970078e-01 1.45569062e+00
  1.17374951e+00 1.60190764e+00 6.07797321e-01 2.62341833e+00]
 [3.

In [6]:

# Shuffles jet constituents
print("Before --->> jetConstituent[0,0:4,0] = ", jetConstituent[0, 0:4, 0])
for i in range(jetConstituent.shape[0]):
    jetConstituent[i] = jetConstituent[i, np.random.permutation(nconstit), :]
print("After --->> jetConstituent[0,0:4,0] = ", jetConstituent[0, 0:4, 0])


from sklearn.model_selection import train_test_split

X = jetConstituent
Y = target
del jetConstituent, target

X_train_val, X_test, Y_train_val, Y_test = train_test_split(
    X, Y, test_size=0.33, random_state=7
)

print(X_train_val.shape, X_test.shape, Y_train_val.shape, Y_test.shape)

Before --->> jetConstituent[0,0:4,0] =  [-177.97363281 -139.30709839 -122.01608276 -112.20430756]
After --->> jetConstituent[0,0:4,0] =  [-112.20430756   -9.28591919   -6.54517365  -34.23769379]
(174200, 30, 16) (85800, 30, 16) (174200, 5) (85800, 5)


In [7]:
print(
    "number of G jets for training/validation: %i"
    % np.sum(np.argmax(Y_train_val, axis=1) == 0)
)
print(
    "number of Q jets for training/validation: %i"
    % np.sum(np.argmax(Y_train_val, axis=1) == 1)
)
print(
    "number of W jets for training/validation: %i"
    % np.sum(np.argmax(Y_train_val, axis=1) == 2)
)
print(
    "number of Z jets for training/validation: %i"
    % np.sum(np.argmax(Y_train_val, axis=1) == 3)
)
print(
    "number of T jets for training/validation: %i"
    % np.sum(np.argmax(Y_train_val, axis=1) == 4)
)


print("number of G jets for testing: %i" % np.sum(np.argmax(Y_test, axis=1) == 0))
print("number of Q jets for testing: %i" % np.sum(np.argmax(Y_test, axis=1) == 1))
print("number of W jets for testing: %i" % np.sum(np.argmax(Y_test, axis=1) == 2))
print("number of Z jets for testing: %i" % np.sum(np.argmax(Y_test, axis=1) == 3))
print("number of T jets for testing: %i" % np.sum(np.argmax(Y_test, axis=1) == 4))


number of G jets for training/validation: 35213
number of Q jets for training/validation: 33694
number of W jets for training/validation: 35083
number of Z jets for training/validation: 35023
number of T jets for training/validation: 35187
number of G jets for testing: 17191
number of Q jets for testing: 16774
number of W jets for testing: 17152
number of Z jets for testing: 17275
number of T jets for testing: 17408


In [8]:
# baseline keras model

njet = X_train_val.shape[0]
nconstit = X_train_val.shape[1]
ntargets = Y_train_val.shape[1]
nfeat = X_train_val.shape[2]


print("#jets = ", njet)
print("#constituents = ", nconstit)
print("#targets = ", ntargets)
print("#features = ", nfeat)

#jets =  174200
#constituents =  30
#targets =  5
#features =  16
